In [1]:
import os 
os.chdir("/home/canyon/Bus-Weather-Impacts")
from src.utils import *
import pandas as pd
import os
import osmnx as ox
import numpy as np
import geopandas as gpd
import networkx as nx
from sklearn.neighbors import KDTree
import numpy as np
pd.options.mode.chained_assignment = None
pd.set_option('display.float_format', '{:.02f}'.format)
from geopy.distance import geodesic
from shapely.geometry import Point
calculated_pair_path = "data/node_pairs.parquet"
pd.set_option('display.max_columns', None)


In [2]:
def compute_distance(row, graph):
    try:
        return nx.shortest_path_length(graph, row['prev_osmid'], row['osmid'], weight='travel_time')
    except nx.NetworkXNoPath:
        return None
    
def compute_path(row, graph):
    try:
        return nx.shortest_path(graph, row['prev_osmid'], row['osmid'], weight='travel_time')
    except nx.NetworkXNoPath:
        return None
    
def compute_euclid_dists(node_pairs, nodes_points):
    nodes_points = nodes_points.to_crs(2263)
    nodes_points["x"] = nodes_points["geometry"].x
    nodes_points["y"] = nodes_points["geometry"].y

    node_pairs = node_pairs.merge(nodes_points, left_on = "osmid", right_on = "osmid", how = "left").merge(nodes_points, right_on = "osmid", left_on = "prev_osmid", how = "left", suffixes = ["curr", "prev"])
    node_pairs["x_diff_sq"] = (node_pairs["xcurr"] - node_pairs["xprev"])**2
    node_pairs["y_diff_sq"] = (node_pairs["ycurr"] - node_pairs["yprev"])**2

    return (node_pairs["x_diff_sq"] + node_pairs["y_diff_sq"]) ** (1/2) / 3.28

def compute_euclid_dists(node_pairs, nodes_points):
    nodes_points = nodes_points.to_crs(2263)
    nodes_points["x"] = nodes_points["geometry"].x
    nodes_points["y"] = nodes_points["geometry"].y

    node_pairs = node_pairs.merge(nodes_points, left_on = "osmid", right_on = "osmid", how = "left").merge(nodes_points, right_on = "osmid", left_on = "prev_osmid", how = "left", suffixes = ["curr", "prev"])
    node_pairs["x_diff_sq"] = (node_pairs["xcurr"] - node_pairs["xprev"])**2
    node_pairs["y_diff_sq"] = (node_pairs["ycurr"] - node_pairs["yprev"])**2

    return (node_pairs["x_diff_sq"] + node_pairs["y_diff_sq"]) ** (1/2) / 3.28

def precalculate_node_pair_distances(node_pair_df, calculated_pair_path, G, nodes):
    try:
        calculated_pairs = pd.read_parquet(calculated_pair_path)
    except:
        print("No pre-calulated pairs found")
        calculated_pairs = pd.DataFrame(columns = ["osmid", "prev_osmid", "distance_osm", "distance_euclid", "shortest_path", "dist_ratio"])

    node_pair_df = node_pair_df.drop_duplicates().dropna()
    node_pair_df = node_pair_df.merge(calculated_pairs, on=["osmid", "prev_osmid"], how="outer")
    pairs_to_calc = node_pair_df.query("distance_euclid.isna()").reset_index(drop = True)
    print(pairs_to_calc.shape)
    
    if pairs_to_calc.shape[0] > 0:
        pairs_to_calc["distance_osm"] = pairs_to_calc.apply(compute_distance, graph=G, axis=1)
        pairs_to_calc["distance_euclid"] = compute_euclid_dists(pairs_to_calc, nodes) 
        pairs_to_calc["shortest_path"] = pairs_to_calc.apply(compute_path, graph=G, axis=1)

        pairs_to_calc["dist_ratio"] = pairs_to_calc["distance_euclid"] / pairs_to_calc["distance_osm"]

        calculated_pairs = pd.concat([calculated_pairs, pairs_to_calc])
        calculated_pairs.to_parquet(calculated_pair_path)
        print(f"Wrote calculated pairs to {calculated_pair_path}")
    else:
        print("No new pairs to calculate")

def prep_buses_nodes(buses_with_nodes, max_distance_to_node):

    buses_with_nodes = buses_with_nodes.sort_values(["trip_id", "timestamp"]).drop_duplicates(subset = ["trip_id", "osmid"],  keep = "first").to_crs(2263)
    buses_with_nodes = buses_with_nodes[["route_short", "timestamp", "trip_id", "next_stop_id", "osmid", "vehicle_id", "distance_to_node", "geometry"]]
    buses_with_nodes["prev_stop_id"] = buses_with_nodes["next_stop_id"].shift(1)
    buses_with_nodes["prev_osmid"] = buses_with_nodes.groupby("trip_id")["osmid"].shift(1)
    buses_with_nodes["next_osmid"] = buses_with_nodes.groupby("trip_id")["osmid"].shift(-1)

    buses_with_nodes["prev_osmid"] = buses_with_nodes["prev_osmid"].astype(float)
    buses_with_nodes["osmid"] = buses_with_nodes["osmid"].astype(float)

    buses_with_nodes = buses_with_nodes.query("distance_to_node < @max_distance_to_node")

    return buses_with_nodes


def tag_feed_with_nodes(buses, tree, nodes, types_to_include = [np.NaN, "traffic_signals", "stop"]):
    nodes = nodes[nodes["highway"].isin(types_to_include)]
    nearest_nodes = tree.query(np.array(buses[['lat', 'lon']]), k=1, return_distance=False)
    buses['nearest_node'] = nearest_nodes.flatten()

    # Map node IDs to OSM IDs
    buses['nearest_osm_id'] = buses['nearest_node'].map(nodes['osmid'])
    buses = buses.merge(nodes, left_on = "nearest_osm_id", right_on = "osmid")
    buses = gpd.GeoDataFrame(buses, geometry='geometry')

    return buses

def get_node_data(place = "New York City, New York, USA"):
    G = ox.graph_from_place(place, network_type='drive')

    G = ox.add_edge_speeds(G)
    G = ox.add_edge_travel_times(G)
    # Convert graph nodes to a DataFrame for KDTree
    nodes = ox.graph_to_gdfs(G, edges=False).reset_index()
    tree = KDTree(nodes[['y', 'x']], metric='euclidean')

    return tree, nodes, G

def prep_coords(df, lat_col, lon_col):

    df['geometry'] = df.apply(lambda row: Point(row[lon_col], row[lat_col]), axis=1)
    gdf = gpd.GeoDataFrame(df, geometry='geometry').set_crs(4326)

    gdf["planar_x"] = gdf.to_crs(2263).geometry.x
    gdf["planar_y"] = gdf.to_crs(2263).geometry.y
    
    gdf = gdf.drop('geometry', axis = 1)
    return gdf

def calculate_distance_to_node(buses_with_nodes):

    out = ((buses_with_nodes.to_crs(2263).geometry.x - buses_with_nodes["planar_x"])**2 + (buses_with_nodes.to_crs(2263).geometry.y - buses_with_nodes["planar_y"])**2)**(1/2)

    return out

def calculate_speeds(prepped_trips, calculated_pair_path = "data/node_pairs.parquet", minimum_time_diff = 45):
    
    node_pair_dists = pd.read_parquet(calculated_pair_path)

    prepped_trips["time_diff_seconds"] = prepped_trips.groupby("trip_id")["timestamp"].diff().dt.total_seconds()
    prepped_trips = prepped_trips.query("time_diff_seconds >= @minimum_time_diff")

    buses_with_distances = prepped_trips.merge(node_pair_dists)
    buses_with_distances["speed_osm"] = (buses_with_distances["distance_osm"] / 1609) / (buses_with_distances["time_diff_seconds"] / 3600)
    buses_with_distances["speed_euclid"] = (buses_with_distances["distance_euclid"] / 1609) / (buses_with_distances["time_diff_seconds"] / 3600)
    
    return buses_with_distances

def explode_edges(row):
    try:
        nodes = row['shortest_path']
        idxs = [int(row['index'])]*(len(nodes)-1)
        osmids = nodes[1:len(nodes)]
        prev_osmids = nodes[0:len(nodes)-1]        
        d = pd.DataFrame(data={'idx':idxs,'from':prev_osmids,'to':osmids})
    except:
        d = pd.DataFrame(data={'idx':[pd.NA],'from':[pd.NA],'to':[pd.NA]})
    
    return d

def get_bus_stops(path = "/home/data/test/cities/C3562/stops.geojson"):
    bus_stops = gpd.read_file(path)
    bus_agencies = ["MTA NYCT", "MTABC", "MTA NYCT,MTABC"]
    bus_stops = bus_stops.query("agency_ids_serviced.isin(@bus_agencies)")[["stop_id", "stop_name", "stop_lat", "stop_lon", "geometry"]].rename({"stop_lat" : "lat", "stop_lon" : "lon"}, axis = 1)
    bus_stops["stop_id"] = [f"MTA_{stop_id}" for stop_id in bus_stops["stop_id"]]
    bus_stops = prep_coords(bus_stops, "lat", "lon")

    return bus_stops

def bus_stops_nodes(bus_stops, tree, nodes):
    stops_with_nodes = tag_feed_with_nodes(bus_stops, tree, nodes)
    stops_with_nodes["dist_to_node"] = calculate_distance_to_node(stops_with_nodes)
    stops_with_nodes = stops_with_nodes.query("dist_to_node < 200")
    stops_with_nodes = stops_with_nodes[["stop_id", "stop_name", "osmid", "dist_to_node"]]

    return stops_with_nodes

def get_stop_pairs(bus_stops, raw_GTFS_path):
    if raw_GTFS_path[:-2] == 'gz':
        gtfs_rt = read_parquet_from_tar_gz(raw_GTFS_path)
    else:
        col_remappings = {"vehicle.trip.trip_id" : "trip_id", "vehicle.timestamp" : "timestamp", "vehicle.position.latitude" : "lat", "vehicle.position.longitude" : "lon", "vehicle.trip.route_id" : "route_short", "vehicle.stop_id" : "next_stop_id", "vehicle.vehicle.id": "vehicle_id"}
        gtfs_rt = pd.read_parquet(raw_GTFS_path).rename(col_remappings, axis = 1)
    gtfs_rt = gtfs_rt.merge(bus_stops, left_on="next_stop_id", right_on = "stop_id", how = "left")
    gtfs_rt = gtfs_rt[["trip_id", "route_short", "timestamp", "next_stop_id"]].sort_values(["trip_id", "timestamp"]).drop_duplicates(["trip_id", "next_stop_id"]).dropna()
    gtfs_rt["prev_stop_id"] = gtfs_rt["next_stop_id"].shift(1)
    #gtfs_rt.loc[gtfs_rt["prev_stop_id"].isna(), 'prev_stop_id'] = gtfs_rt["origin_id"]

    stop_pairs = gtfs_rt[["prev_stop_id", "next_stop_id"]]

    stop_pairs = stop_pairs.merge(bus_stops[["stop_id", "stop_name", "osmid"]], left_on = "prev_stop_id", right_on = "stop_id").merge(bus_stops[["stop_id", "stop_name", "osmid"]], left_on = "next_stop_id", right_on = "stop_id", suffixes = ["_prev", "_next"]).rename({"osmid_next" : "osmid", "osmid_prev" : "prev_osmid"}, axis = 1)
    stop_pairs["osmid"] = stop_pairs["osmid"].astype(int)
    stop_pairs["prev_osmid"] = stop_pairs["prev_osmid"].astype(int)

    return stop_pairs.drop_duplicates()

def get_pair_paths(stop_pairs, G, nodes, calculated_pair_path = "data/node_pairs.parquet"):
    precalculate_node_pair_distances(stop_pairs[["osmid", "prev_osmid"]], calculated_pair_path=calculated_pair_path, G = G, nodes = nodes)
    node_pair_dists = pd.read_parquet(calculated_pair_path)
    stop_pairs = stop_pairs[["osmid", "prev_osmid", "next_stop_id", "prev_stop_id", "stop_name_prev", "stop_name_next"]].merge(node_pair_dists)
    
    return stop_pairs

def full_process_stops(tree, nodes, G, GTFS_PATH, calculated_pair_path = "data/node_pairs.parquet", stops_path = "/home/data/test/cities/C3562/stops.geojson"):
    bus_stops = get_bus_stops(stops_path)
    bus_stops = bus_stops_nodes(bus_stops, tree, nodes)
    stop_pairs = get_stop_pairs(bus_stops, GTFS_PATH)
    stop_pairs = get_pair_paths(stop_pairs, G, nodes, calculated_pair_path)

    return stop_pairs[["next_stop_id", "prev_stop_id",  "stop_name_prev", "stop_name_next", "shortest_path"]].rename({"shortest_path" : "shortest_path_stops"}, axis = 1)

def check_in_bus_path(row):
    if isinstance(row["shortest_path_stops"], (list, np.ndarray)):
        return row["osmid"] in row["shortest_path_stops"]
    return False

def process_gtfs_rt_main(tree, nodes, G, gtfs_path, calculated_pair_path, out_path, stops_with_paths = None, max_distance_to_node = 100):

    print("Preprocssing bus data")
    if gtfs_path[:-2] == 'gz':
        buses = read_parquet_from_tar_gz(gtfs_path)
    else:
        col_remappings = {"vehicle.trip.trip_id" : "trip_id", "vehicle.timestamp" : "timestamp", "vehicle.position.latitude" : "lat", "vehicle.position.longitude" : "lon", "vehicle.trip.route_id" : "route_short", "vehicle.stop_id" : "next_stop_id", "vehicle.vehicle.id": "vehicle_id"}
        buses = pd.read_parquet(gtfs_path).rename(col_remappings, axis = 1)
    buses = prep_coords(buses, 'lat', 'lon')

    print("Tagging bus locations with nodes")
    buses_with_nodes = tag_feed_with_nodes(buses, tree, nodes)
    buses_with_nodes["distance_to_node"] = calculate_distance_to_node(buses_with_nodes)


    print("Calculating distance pairs")
    prepped_trips = prep_buses_nodes(buses_with_nodes, max_distance_to_node)

    if stops_with_paths is not None:
        print(prepped_trips.shape)
        prepped_trips = prepped_trips.merge(stops_with_paths)
        prepped_trips["in_bus_path"] = prepped_trips.apply(check_in_bus_path, axis=1)
        prepped_trips = prepped_trips.query("in_bus_path")
        print(prepped_trips.shape)


    node_pair_df = prepped_trips[["osmid", "prev_osmid"]]

    precalculate_node_pair_distances(node_pair_df, calculated_pair_path, G, nodes)
    buses_with_speeds = calculate_speeds(prepped_trips).reset_index()

    print("Exploding edges")
    dfs = []
    for i,row in buses_with_speeds.iterrows():
        dfs.append(explode_edges(row))

    segment_speeds = pd.concat(dfs)
    bus_speed_segemented = buses_with_speeds.merge(segment_speeds, left_on = "index", right_on = "idx").drop(["index"], axis = 1)

    print("Writing to parquet")
    bus_speed_segemented.to_parquet(out_path)

In [ ]:
read_parquet_from_tar_gz("https://urbantech-public.s3.amazonaws.com/DO-NOT-DELETE-BUSOBSERVATORY-PUBLIC-DATASET/one-system-day.tar.gz")

In [5]:
#GTFS_PATH = "https://urbantech-public.s3.amazonaws.com/DO-NOT-DELETE-BUSOBSERVATORY-PUBLIC-DATASET/one-system-day.tar.gz"
GTFS_PATH = "/home/data/bus-weather/raw_bus_gtfs_rt_202230917_20230930.parquet"
CALCULATED_PAIR_PATH = "data/node_pairs.parquet"
OUT_PATH = "data/buses_with_segmented_storm.parquet"
STOPS_PATH = "/home/data/test/cities/C3562/stops.geojson"

In [6]:
tree, nodes, G = get_node_data()

KeyboardInterrupt: 

In [ ]:
stops_with_paths = full_process_stops(tree, nodes, G, GTFS_PATH, calculated_pair_path = CALCULATED_PAIR_PATH, stops_path = "/home/data/test/cities/C3562/stops.geojson")

In [ ]:
process_gtfs_rt_main(tree, nodes, G, GTFS_PATH, CALCULATED_PAIR_PATH, OUT_PATH, stops_with_paths)

Preprocssing bus data


: 

In [ ]:
new_buses = gpd.read_parquet(OUT_PATH)
old_buses = gpd.read_parquet("/home/canyon/Bus-Weather-Impacts/data/buses_with_segmented.parquet")

In [ ]:
quantiles = [0.01, 0.1, 0.25, 0.5, 0.75, 0.9, .99]


In [ ]:
new_buses[["speed_osm", "speed_euclid", "dist_ratio"]].describe(percentiles=quantiles)

In [ ]:
old_buses[["speed_osm", "speed_euclid", "dist_ratio"]].describe(percentiles=quantiles)

In [ ]:
bus_speed_segemented.query("speed_osm < 70").query("`from` == 4209661118 & to == 4209661121.00")['speed_osm'].hist()

In [ ]:
segment_speeds.to_parquet("segments_test.parquet")

In [ ]:
prepped_trips["time_diff_seconds"] = prepped_trips.groupby("trip_id")["timestamp"].diff().dt.total_seconds()